In [1]:
import sys
import pandas as pd

sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')

from Pipeline.recommender_tuning import hypertune
from Pipeline.data_extraction import get_dataframes
from Pipeline.matrices_creation import get_URM_split_val, create_URM
from DressipiChallenge.Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
from Pipeline.optuna_utils import Integer, Real, Categorical

In [3]:
item_features_df, train_sessions_df, train_purchases_df, test_sessions_df, candidate_items_df = get_dataframes("../../")

URM_train, URM_val_views, URM_val_purch, mapped_items_to_ignore, mapped_val_sessions_arr, val_session_mapping, item_mapping = get_URM_split_val(
    item_features_df=item_features_df,
    train_purchases_df=train_purchases_df,
    train_sessions_df=train_sessions_df,
    val_start_ts='2021-05-01',
    val_end_ts='2021-06-01',
    unique_interactions=True,
    abs_half_life=365,
    cyclic_decay=False,
    purch_weight=1,
    view_weight=0.5,
    score_graph=False,
)

CSVs read


In [3]:
multvae = MultVAERecommender_OptimizerMask(URM_train)

MultVAERecommender: URM Detected 477 ( 2.0%) items with no interactions.


In [4]:
best_hyp = {'epochs': 1, 'learning_rate': 1.4396337313314092e-05, 'l2_reg': 1.9803558930283507e-06, 'dropout': 0.7726522034872121, 'total_anneal_steps': 388484, 'anneal_cap': 0.266817776801911, 'batch_size': 128, 'encoding_size': 227, 'next_layer_size_multiplier': 6, 'max_n_hidden_layers': 2, 'max_parameters': 1750000000.0}

In [5]:
multvae.fit(**best_hyp)

MultVAERecommender: Architecture: [227, 1362, 23691]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_17336/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_17336/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 1. Elapsed time 6.11 min
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_17336/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Terminating at epoch 1. Elapsed time 6.14 min
MultVAERecommender: Loading model from file './result_experiments/__Temp_MultVAERecommender_17336/_best_model'
INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_17336/_best_model/.session/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_17336/'


In [42]:
from scipy import sparse

X = multvae.URM_train[0]

if sparse.isspmatrix(X):
    X = X.toarray()
X = X.astype('float32')

feed_dict = {multvae.vae.input_ph: X}

In [43]:
output = multvae.sess.graph.get_tensor_by_name(multvae.vae.embeddings_name)
output_values = multvae.sess.run(output, feed_dict=feed_dict)
output_values

array([[ 0.09989955, -0.01976695, -0.07885621,  0.08938596, -0.0844129 ,
        -0.10481874, -0.14289923,  0.02802889, -0.10555656, -0.10308403,
        -0.10784927,  0.01435503,  0.12524645, -0.09677149,  0.00658246,
         0.05537899, -0.03680438,  0.07453355,  0.12719691, -0.07135529,
        -0.09758721, -0.12293227, -0.08562053,  0.01820915, -0.05475412,
        -0.06142215,  0.04265151,  0.15643267, -0.05650007, -0.03106585,
        -0.07743905,  0.01486019, -0.1115837 ,  0.10318891, -0.03090213,
         0.03481588,  0.04729668,  0.15693668,  0.07278181,  0.17468531,
        -0.06920554, -0.16824917, -0.03436874,  0.05336907, -0.17737909,
        -0.04835016, -0.03374434,  0.1633553 ,  0.03540198,  0.09597493,
         0.11361274, -0.13205245,  0.03426874, -0.11482158,  0.09487895,
         0.09321387,  0.09976232,  0.01830675,  0.06262195,  0.06669343,
         0.02472619, -0.06360423, -0.02154977,  0.12757298,  0.09411594,
        -0.15562224, -0.03077425,  0.15879841, -0.0

In [56]:
import numpy as np

from DressipiChallenge.Pipeline.data_splitting import split_dataframes_val
from DressipiChallenge.Pipeline.matrices_creation import create_csr_matrix

def get_vae_embeddings(folder_path,  batch_size=100, save_path="./session_embeddigs.parquet", is_test = False):
    # ohhh the misery

    val_end_ts = '2021-07-01' if is_test else '2021-06-01'
    val_start_ts = '2021-06-01' if is_test else '2021-05-01'

    weighted_train_set_df, val_purch_df, val_views_df, train_session_mapping, \
    val_session_mapping, item_mapping, mapped_items_to_ignore, mapped_val_sessions_arr = split_dataframes_val(
        train_sessions_df, train_purchases_df, item_features_df,
        val_start_ts=val_start_ts, val_end_ts=val_end_ts,
        unique_interactions=True, view_weight=1, purch_weight=0, abs_half_life=None,
        cyclic_decay=False, score_graph=False
    )

    URM_train = create_csr_matrix(weighted_train_set_df, len(train_session_mapping), len(item_mapping))
    URM_val_views = create_csr_matrix(val_views_df, len(val_session_mapping), len(item_mapping))

    multvae = MultVAERecommender_OptimizerMask(URM_train)
    multvae.load_model(folder_path)

    embeddings_train = np.zeros(shape=(len(train_session_mapping), multvae.p_dims[0]))

    processed = 0

    for _, st_idx in enumerate(range(0, 200, batch_size)):

        end_idx = min(st_idx + batch_size, len(train_session_mapping))
        X = URM_train[st_idx:end_idx]

        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')

        output = multvae.sess.graph.get_tensor_by_name(multvae.vae.embeddings_name)
        feed_dict = {multvae.vae.input_ph: X,
                     multvae.vae.is_training_ph: 1}
        output_values = multvae.sess.run(output, feed_dict=feed_dict)
        embeddings_train[st_idx:end_idx] = output_values

        processed += end_idx-st_idx
        print("Processed {0} train user.".format(processed))

    df_train = pd.DataFrame(embeddings_train)
    df_train.to_csv(save_path, index=False)

    print("Saved train. Continuning with test.")
    # Partial train to not lose data...

    embeddings_test = np.zeros(shape=(len(val_session_mapping), multvae.p_dims[0]))

    processed = 0

    for _, st_idx in enumerate(range(0, 200, batch_size)):

        end_idx = min(st_idx + batch_size, len(val_session_mapping))
        X = URM_val_views[st_idx:end_idx]

        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')

        output = multvae.sess.graph.get_tensor_by_name(multvae.vae.embeddings_name)
        feed_dict = {multvae.vae.input_ph: X,
                     multvae.vae.is_training_ph: 1}
        output_values = multvae.sess.run(output, feed_dict=feed_dict)
        embeddings_test[st_idx:end_idx] = output_values

        processed += end_idx-st_idx
        print("Processed {0} test user.".format(processed))

    df_test = pd.DataFrame(embeddings_test)
    
    # merge both

    val_inverse = {k:v for v,k in val_session_mapping.items()}
    train_inverse = {k:v for v,k in train_session_mapping.items()}

    df_train.insert(loc=0, column="session_id", value=[train_inverse[x] for x in np.arange(len(df_train))])
    df_test.insert(loc=0, column="session_id", value=[val_inverse[x] for x in np.arange(len(df_test))])

    df_test = pd.concat([df_train, df_test]).reset_index(drop=True)
    df_test.to_parquet(save_path, index=False)

    print("Saved test.")

    return df_test


In [57]:
get_vae_embeddings("./owo.h5")

MultVAERecommender: URM Detected 600 ( 2.5%) items with no interactions.
MultVAERecommender: Loading model from file './owo.h5MultVAERecommender'
INFO:tensorflow:Restoring parameters from ./owo.h5MultVAERecommender/.session/session
MultVAERecommender: Loading complete
Processed 100 train user.
Processed 200 train user.
Saved train. Continuning with test.
Processed 100 test user.
Processed 200 test user.
Saved test.


,session_id,0,1,2,3,4,5,6,7,8,...,217,218,219,220,221,222,223,224,225,226
0,3,-0.079773,0.452049,-0.250728,-0.070084,0.200194,-1.441675,-1.983052,0.233127,-0.151445,...,0.179735,-1.172273,0.070427,-0.116233,0.128644,0.349428,1.009601,-0.071348,-0.129234,1.510903
1,13,-1.779117,-1.147733,-0.274971,0.682536,-1.149518,0.595269,0.810294,-0.690703,-0.197307,...,-0.510440,-1.199814,-0.020504,0.353472,-0.771306,-0.277225,0.149223,-0.062017,0.558752,0.956635
2,18,-1.060449,-0.557475,-1.038367,0.971215,-1.896285,-0.153937,-0.212554,-1.468928,-0.300046,...,-2.169792,-0.737722,-1.433205,0.835977,0.021403,-0.574306,-0.187727,0.627984,-0.909470,0.149374
3,19,1.870649,-1.011784,1.193372,1.227687,-0.078799,-1.017636,1.449390,0.878878,-1.282964,...,0.322996,-0.184029,0.690260,0.265269,0.176457,1.338422,1.385663,-0.717638,2.053130,0.923624
4,24,-0.437075,-0.820023,0.692018,0.030970,0.645132,-0.841275,-0.541000,0.617516,-0.461960,...,0.423458,0.065053,1.738762,-1.165564,0.750926,1.222507,0.559896,-0.775158,1.005858,2.995511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,4439680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999996,4439823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999997,4439898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999998,4439949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
pd.read_csv("./session_embeddigs.csv")

,session_id,0,1,2,3,4,5,6,7,8,...,217,218,219,220,221,222,223,224,225,226
0,3,-0.079773,0.452049,-0.250728,-0.070084,0.200194,-1.441675,-1.983052,0.233127,-0.151445,...,0.179735,-1.172273,0.070427,-0.116233,0.128644,0.349428,1.009601,-0.071348,-0.129234,1.510903
1,13,-1.779117,-1.147733,-0.274971,0.682536,-1.149518,0.595269,0.810294,-0.690703,-0.197307,...,-0.510440,-1.199814,-0.020504,0.353472,-0.771306,-0.277225,0.149223,-0.062017,0.558752,0.956635
2,18,-1.060449,-0.557475,-1.038367,0.971215,-1.896285,-0.153937,-0.212554,-1.468928,-0.300046,...,-2.169792,-0.737722,-1.433205,0.835977,0.021403,-0.574306,-0.187727,0.627984,-0.909470,0.149374
3,19,1.870649,-1.011784,1.193372,1.227687,-0.078799,-1.017636,1.449390,0.878878,-1.282964,...,0.322996,-0.184029,0.690260,0.265269,0.176457,1.338422,1.385663,-0.717638,2.053130,0.923624
4,24,-0.437075,-0.820023,0.692018,0.030970,0.645132,-0.841275,-0.541000,0.617516,-0.461960,...,0.423458,0.065053,1.738762,-1.165564,0.750926,1.222507,0.559896,-0.775158,1.005858,2.995511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,4439680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999996,4439823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999997,4439898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999998,4439949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


: 

In [5]:
session_embeddings_on_train = pd.read_csv("../../Dataset/embeddings_session.csv")

In [12]:
session_val = session_embeddings_on_train.loc[session_embeddings_on_train.session_id.isin(list(val_session_mapping.keys()))]
session_val.to_csv("../../Dataset/session_val_embeddings.csv", index=False)

In [14]:
del session_embeddings_on_train
del session_val

In [13]:
session_embeddings_on_val = pd.read_csv("../../Dataset/embeddings_session_test.csv")

In [18]:
session_train_test = session_embeddings_on_val.loc[~session_embeddings_on_val.session_id.isin(list(val_session_mapping.keys()))]
session_test = session_train_test.loc[~session_train_test.session_id.isin(list(train_sessions_df.session_id.unique()))]
session_test.to_csv("../../Dataset/session_test_embeddings.csv", index=False)